In [15]:
import sqlite3
import os, sys

data_dir = '/Users/xlx/Downloads/graph-data'
db_file = os.path.join(data_dir, 'AcademicGraph.db')
schema_file = os.path.join(data_dir, "readme.txt")

In [130]:
def read_schema_file(schema_file, table_name):
    field_list = []
    with open(schema_file, 'rt') as fh:
        
        for line in fh:
            if line.strip() == table_name:
                break
        
        cl = fh.readline().strip()
        while len(cl) > 0:
            tmp = cl.split('\t', maxsplit=1)
            field_list.append( tmp[1].replace(' ', '_').replace('(', '').replace(')', '') )
            cl = fh.readline().strip()

    if not field_list: #EOF reached
        print("reached the end of {}, no {} found".format(schema_file, table_name))

    return field_list

In [146]:
print( read_schema_file(schema_file, 'Journals') )
print( read_schema_file(schema_file, 'ConferenceSeries') )
print( read_schema_file(schema_file, 'ConferenceInstances') )

['Journal_ID', 'Journal_name']
['Conference_series_ID', 'Short_name_abbreviation', 'Full_name']
['Conference_series_ID', 'Conference_instance_ID', 'Short_name_abbreviation', 'Full_name', 'Location', 'Official_conference_URL', 'Conference_start_date', 'Conference_end_date', 'Conference_abstract_registration_date', 'Conference_submission_deadline_date', 'Conference_notification_due_date', 'Conference_final_version_due_date']


In [147]:
def creat_table(table_name, db_file, schema_file=schema_file, data_file=None):
	field_list = read_schema_file(schema_file, table_name)

	conn = sqlite3.connect(db_file)
	c = conn.cursor()
	# flush+delete table if exist 
	c.execute('DROP TABLE IF EXISTS ' + table_name) #want a tuple with >1 elements
	conn.commit()
    # create table w. schema
    ## CREATE TABLE stocks (date text, trans text, symbol text, qty real, price real)
	schema_str = ', '.join([f+' text' for f in field_list])
	print(schema_str)
	cmd = "CREATE TABLE {0} ({1})".format(table_name, schema_str)
	c.execute(cmd )
	conn.commit()
    
    # now read some data
	readcnt = 0
	if data_file:
		try:
			cur_data_list = []
			value_holder = ','.join( ['?' for f in field_list] )
			cmd = 'INSERT INTO ' + table_name + ' VALUES (' + value_holder + ')'
			print(cmd)
            
			for s in open(data_file, 'r'):
				cur_tuple = tuple(s.strip().split('\t'))
				#assert len(field_list)==len(cur_tuple), "number of fields mismatch! - {} ".format(cur_tuple)
				if len(field_list) > len(cur_tuple): #pad emtpy to the end
					num_missing = len(field_list) - len(cur_tuple)
					for i in range(num_missing):
						cur_tuple = cur_tuple + ("",)
				elif len(field_list) > len(cur_tuple):
					"number of fields error! - {} ".format(cur_tuple)
				cur_data_list.append( cur_tuple )
				readcnt +=1 
                
				if readcnt % 5000 == 0:                
					c.executemany(cmd, cur_data_list)
					conn.commit()
					cur_data_list = []
					print("read {} lines".format(readcnt))
				
			# finish the final write 
			c.executemany(cmd, cur_data_list)
			conn.commit()
				
		except OSError:
			print("data file '{}' read error!".format(data_file))

    # simple counting check
	cmd = 'SELECT COUNT(*) FROM {}'.format(table_name)
	datacnt = c.execute(cmd).fetchone()[0]
	print( "read {} rows from file, {} found in db ".format(readcnt, datacnt) )
	conn.close()

In [148]:
table_name_list = ['ConferenceSeries', 'Journals', 'FieldsOfStudy', 'ConferenceInstances']
for table_name in table_name_list:
    data_file = os.path.join(data_dir, table_name+'.txt')
    creat_table(table_name, db_file, schema_file=schema_file, data_file=data_file)

Conference_series_ID text, Short_name_abbreviation text, Full_name text
INSERT INTO ConferenceSeries VALUES (?,?,?)
read 1275 rows from file, 1275 found in db 
Journal_ID text, Journal_name text
INSERT INTO Journals VALUES (?,?)
read 5000 lines
read 10000 lines
read 15000 lines
read 20000 lines
read 23568 rows from file, 23568 found in db 
Field_of_study_ID text, Field_of_study_name text
INSERT INTO FieldsOfStudy VALUES (?,?)
read 5000 lines
read 10000 lines
read 15000 lines
read 20000 lines
read 25000 lines
read 30000 lines
read 35000 lines
read 40000 lines
read 45000 lines
read 50000 lines
read 53834 rows from file, 53834 found in db 
Conference_series_ID text, Conference_instance_ID text, Short_name_abbreviation text, Full_name text, Location text, Official_conference_URL text, Conference_start_date text, Conference_end_date text, Conference_abstract_registration_date text, Conference_submission_deadline_date text, Conference_notification_due_date text, Conference_final_version_due_

In [137]:
creat_table('Journals', db_file, schema_file=schema_file, data_file=None)

data_file = os.path.join(data_dir, 'Journals'+'.txt')
creat_table('Journals', db_file, schema_file=schema_file, data_file=data_file)

Journal_ID text, Journal_name text
read 0 rows from file, 0 found in db 
Journal_ID text, Journal_name text
INSERT INTO Journals VALUES (?,?)
read 5000 lines
read 10000 lines
read 15000 lines
read 20000 lines
read 23568 rows from file, 23568 found in db 
